In [64]:
#PRIMERA PARTE

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from urllib.request import urlopen
!pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print("Libraries imported.")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.1
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.1
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [67]:
#Vamos a extraer los datos de la dirección que nos suministra Coursera
url ="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page=urlopen(url).read().decode("utf-8")
soup = BeautifulSoup(page,"html.parser")

wikipedia_table =soup.body.table.tbody

#Observamos si los resultados se han importado correctamente
wikipedia_table

<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</t

In [4]:
#A continuación, vamos a extraer los datos de la tabla importada
#a un marco de datos

def get_cell(element):
    cells = element.find_all("td")
    row=[]
    for cell in cells:
        if cell.a:
            if(cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
    return row
def get_row():
    data=[]
    for tr in wikipedia_table.find_all("tr"):
        row=get_cell(tr)
        if len(row) !=3:
            continue
        data.append(row)
    return data
data = get_row()
columns = ["PostalCode", "Borough", "Neighbourhood"]
df_wiki=pd.DataFrame(data,columns=columns)
df_wiki.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
#Comprobamos que el número de elementos de cada variable coincide
df_wiki.count()

PostalCode       180
Borough          180
Neighbourhood    180
dtype: int64

In [6]:
#Otra forma sería empleando la función shape, aunque no sería tan 
#detallado, ya que no mostraría las variables
df_wiki.shape

(180, 3)

In [7]:
#A continuación, vamos a proceder a la limpieza de los datos
df_unavailable=df_wiki[df_wiki.Borough!="Not assigned"]
df_unavailable=df_unavailable.sort_values(by=["PostalCode","Borough"])

df_unavailable.reset_index(inplace=True)
df_unavailable.drop("index", axis=1, inplace=True)

df_unavailable.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df_postcodes = df_unavailable["PostalCode"]
df_postcodes.drop_duplicates(inplace=True)
df_37= pd.DataFrame(df_postcodes)
df_37["Borough"]="";
df_37["Neighbourhood"]="";

df_37.reset_index(inplace=True)
df_37.drop("index", axis=1, inplace=True)
df_unavailable.reset_index(inplace=True)
df_unavailable.drop("index", axis=1, inplace=True)

for i in df_37.index:
    for j in df_unavailable.index:
        if df_37.iloc[i,0]==df_unavailable.iloc[j,0]:
            df_37.iloc[i,1] = df_unavailable.iloc[j,1]
            df_37.iloc[i,2] = df_37.iloc[i,2]+","+df_unavailable.iloc[j,2]

for i in df_37.index:
    s=df_37.iloc[i,2]
    if s[0] == ",":
        s=s[1:]
    df_37.iloc[i,2]=s


In [9]:
#Veamos la muestra una vez filtrada (eliminadas duplicidades,...)
df_37.count()

PostalCode       103
Borough          103
Neighbourhood    103
dtype: int64

In [10]:
#Otra forma de ver la muestra
df_37.shape

(103, 3)

In [11]:
df_37.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [65]:
#2 PARTE

In [12]:
#Cargamos los datos geográficos
df_37["Latitude"]="0";
df_37["Longitude"]="0";

df_geograp=pd.read_csv("https://cocl.us/Geospatial_data")
df_geograp.columns=["Postcade","Latitude","Longitude"]

In [13]:
#Mostramos los datos geográficos
df_geograp

,Postcade,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [14]:
#Los añadimos a la tabla anterior
for i in df_37.index:
    for j in df_geograp.index:
        if df_37.iloc[i,0] == df_geograp.iloc[j,0]:
            df_37.iloc[i,3] = df_geograp.iloc[j,1]
            df_37.iloc[i,4] = df_geograp.iloc[j,2]

In [15]:
#Mostrar los resultados
df_37.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [16]:
df_37.shape
df_37.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [66]:
#PARTE 3

In [17]:
#Vamos a empezar a crear un mapa de Toronto, para
#agrupar por clusters los vecindarios
#Empezamos calculando la latitud y longitud de Toronto

address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent="ny_explorer", timeout=30)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


In [18]:
#Importamos bibliotecas de renderización

from pandas.io.json import json_normalize
import json

In [19]:
#Vamos a crear un mapa de Toronto, usando los valores
#obtenidos anteriormente

mapadetoronto=folium.Map(location=[latitude,longitude],zoom_start=10)

#A continuación, vamos a adherir marcas en el mapa de
#Toronto, basado en los valores de latitud y longitud
#de Postcode
for lat, lng, neighbourhood in zip(df_37["Latitude"],
                                  df_37["Longitude"],
                                  df_37["Neighbourhood"]):
    label="{}".format(neighbourhood)
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color="orange",
        fill=True,
        fill_color="#D62598",
        fill_opacity=0.7,
        parse_html=False).add_to(mapadetoronto)

In [20]:
mapadetoronto

In [28]:
#A continuación, vamos a definir las credenciales de
#Foursquare:(client_id,client_secret,...)

CLIENT_ID= "FDRO04QUOEBWQRFU4SBSRMUVH15OY1UBVHIHB0W1W35BJMRZ"
CLIENT_SECRET= "5JZYFEW21KGCQQIPG0LYY0OOH3RU3G40GGAZOJDCRGLLHVPO"
VERSION="20180604"
LIMIT=30
print("Your credentials:")
print("CLIENT_ID: " + CLIENT_ID)
print("CLIENT_SECRET:" + CLIENT_SECRET)

Your credentials:
CLIENT_ID: FDRO04QUOEBWQRFU4SBSRMUVH15OY1UBVHIHB0W1W35BJMRZ
CLIENT_SECRET:5JZYFEW21KGCQQIPG0LYY0OOH3RU3G40GGAZOJDCRGLLHVPO


In [46]:
LIMIT = 5
radius = 500

Top5lugares = [] # Prepara una lista para almacenar la solicitud de Foursquare

for neighbourhood, latitude, longitude in zip(df_37.Neighbourhood, df_37.Latitude, df_37.Longitude):
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
    
    data = requests.get(url).json()
    
    # Usa la función len() para comprobar si hay datos sin elementos (cuando esta toma el valor 0, significa que no hay elementos)
    length = len(data['response']['groups'][0]['items'])
    if length == 0:
        continue # Saltar de fila si no se encuentran items
    else:
        venue = data['response']['groups'][0]['items'][0]['venue']
        
        # Extraer información 'venue'
        name = venue['name']
        lat = venue['location']['lat']
        lon = venue['location']['lng']
        cat = venue['categories'][0]['name']
        
        Top5lugares.append([(neighbourhood, latitude, longitude, name, lat, lon, cat)])


In [47]:
#Crear un marco de datos basados en "Top5lugares"

temp = pd.DataFrame(x for row in Top5lugares for x in row)
temp.columns = ['Neighbourhood','N_Latitude','N_Longitude','Venue','V_Latitude','V_Longitude','category']
temp.head()

,Neighbourhood,N_Latitude,N_Longitude,Venue,V_Latitude,V_Longitude,category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
4,Cedarbrae,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant


In [48]:

print("{} localizaciones cercanas han sido descargadas de {} vecindarios.".format(len(temp.Venue), len(df_37.Neighbourhood)))
        

100 localizaciones cercanas han sido descargadas de 103 vecindarios.


In [54]:
cat = pd.get_dummies(temp.category) # vamos a crear a través del metodo de one encoding, variables tipo dummies 
df_dummy = pd.concat([temp[['Neighbourhood']], cat], axis=1) # vamos a combinar el vecindario y las tablas categorías que hemos creado gracias a las variables dummies
df_dummy.head()


,Neighbourhood,Airport,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Burger Joint,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Fast Food Restaurant,Field,Garden,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hakka Restaurant,Hockey Arena,Hotel,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Martial Arts School,Massage Studio,Mexican Restaurant,Motel,Neighborhood,Park,Pharmacy,Pizza Place,Playground,Pool,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Sports Bar,Theme Restaurant,Toy / Game Store,Trail,Warehouse Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Woburn,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Cedarbrae,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
#importa k-means para la etapa de agrupamiento
from sklearn.cluster import KMeans

In [55]:
df_02 = df_dummy.drop('Neighbourhood', axis=1)

n_group = 6 # Vamos a agrupar los 6 vecindarios en 6 grupos

# ejecuta los agrupamientos k-means
kmeans = KMeans(n_clusters=n_group, random_state=0).fit(df_02)

#Comprueba las etiquetas de agrupamiento generadas para cada fila en el marco de datos
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2], dtype=int32)

In [56]:
#Añade las etiquetas de agrupamiento
df_dummy.insert(1, 'label', kmeans.labels_)

In [57]:

df_dummy.head() #"Etiqueta" generadas por k-means se han incluido en el marco de datos

,Neighbourhood,label,Airport,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Burger Joint,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Fast Food Restaurant,Field,Garden,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hakka Restaurant,Hockey Arena,Hotel,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Martial Arts School,Massage Studio,Mexican Restaurant,Motel,Neighborhood,Park,Pharmacy,Pizza Place,Playground,Pool,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Sports Bar,Theme Restaurant,Toy / Game Store,Trail,Warehouse Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Woburn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Cedarbrae,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [62]:
# Vamos a proceder a la unión de df_37 y df_dummy

toronto_unido= pd.merge(df_37, df_dummy, on='Neighbourhood', how='right')
toronto_unido.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,label,Airport,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Burger Joint,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Fast Food Restaurant,Field,Garden,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Hakka Restaurant,Hockey Arena,Hotel,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Martial Arts School,Massage Studio,Mexican Restaurant,Motel,Neighborhood,Park,Pharmacy,Pizza Place,Playground,Pool,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Sports Bar,Theme Restaurant,Toy / Game Store,Trail,Warehouse Store,Yoga Studio
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845,-79.1605,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M1G,Scarborough,Woburn,43.771,-79.2169,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
# Vamos a crear el mapa
mapa_agrupamientos_de_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#Definamos el esquema del color para los agrupamientos
x = np.arange(n_group)
ys = [i + x + (i*x)**2 for i in range(n_group)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Añadimos marcas al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_unido['Latitude'], toronto_unido['Longitude'], toronto_unido['Neighborhood'], toronto_unido['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapa_agrupamientos_de_Toronto)
       
mapa_agrupamientos_de_Toronto